# [4th Plan] RAG 챗봇 AWS 배포 및 Django 웹 구축 계획

본 문서는 `chatbot_graph_V8_FINAL.py`로 구현된 RAG 기반 법률 챗봇을 AWS EC2 서버에 배포하고, Django를 이용해 웹 인터페이스를 구축하기 위한 상세 계획입니다.

---

## 1. 개요 및 목표

### 주요 목표
1. **Django 웹 서버 구축**: 사용자 입력을 받아 RAG 모델과 통신하는 웹 애플리케이션 개발
2. **챗봇 모델 통합**: 기존 `chatbot_graph_V8_FINAL.py`를 Django 서비스 레이어에 통합
3. **AWS EC2 배포**: 클라우드 환경에서 24시간 서비스 가능한 상태로 배포 (Nginx + Uvicorn)

### 시스템 아키텍처
- **Frontend**: HTML/CSS/JS (Django Templates)
- **Backend**: Django (Async Views) + Uvicorn (ASGI Server)
- **AI Logic**: LangGraph (`LegalRAGBuilder`) + OpenAI + Qdrant
- **Infrastructure**: AWS EC2 (Ubuntu), Nginx (Reverse Proxy)

## 2. Django 프로젝트 내 챗봇 통합 (Local 개발)

### 2.1 프로젝트 구조 설정
기존 `a_team` 디렉토리 내에 Django 프로젝트를 생성하거나, 새로운 루트에서 시작하여 스크립트를 import 할 수 있게 설정합니다.

```bash
# 가상환경 활성화 상태에서 진행
pip install django djangorestframework django-environ uvloop uvicorn

# 프로젝트 생성 (예: legal_chat)
django-admin startproject legal_chat
cd legal_chat

# 앱 생성 (예: bot)
python manage.py startapp bot
```

### 2.2 디렉토리 구조 예시
```text
legal_chat/
├── manage.py
├── legal_chat/
│   ├── settings.py
│   ├── asgi.py   <-- 중요 (비동기 처리를 위해 wsgi.py 대신 사용)
│   └── ...
├── bot/
│   ├── views.py
│   ├── services.py  <-- 챗봇 로직 연결 (Wrapper)
│   └── ...
└── scripts/         <-- 기존 스크립트 폴더 복사 또는 심볼릭 링크
    └── architectures/
        └── chatbot_graph_V8_FINAL.py
```

### 2.3 챗봇 서비스 레이어 구현 (`bot/services.py`)
Django View에서 직접 비즈니스 로직을 다루기보다, `Service` 클래스를 만들어 챗봇 인스턴스를 관리합니다. 
LangGraph 모델이 비동기(Async)로 동작하므로, Django도 Async View를 사용하는 것이 효율적입니다.

```python
# bot/services.py
import sys
from pathlib import Path
from django.conf import settings

# 스크립트 경로 추가 (필요시)
BASE_DIR = Path(__file__).resolve().parent.parent
SCRIPTS_PATH = BASE_DIR / 'scripts' / 'architectures'
sys.path.append(str(SCRIPTS_PATH))

from chatbot_graph_V8_FINAL import LegalRAGBuilder, Config, AgentState

class ChatbotService:
    _instance = None
    _builder = None
    _graph = None

    @classmethod
    async def get_instance(cls):
        if cls._instance is None:
            cls._instance = ChatbotService()
            await cls._instance._initialize()
        return cls._instance

    async def _initialize(self):
        # Config 설정 (환경변수는 settings.py 또는 .env에서 로드된다고 가정)
        config = Config()
        # builder 초기화
        self._builder = LegalRAGBuilder(config)
        
        # 그래프 생성: Builder 내부의 build() 메서드 사용
        # build() 메서드 내부에서 _init_infrastructure() 호출 및 노드/엣지 연결을 모두 수행함
        self._graph = self._builder.build()
        
        print("Chatbot Graph Initialized successfully.")

    async def get_response(self, user_message: str):
        if not self._graph:
            # 그래프 초기화 로직 수행
            pass
        
        inputs = {"user_query": user_message, "messages": [("user", user_message)]}
        
        # ainvoke 호출
        result = await self._graph.ainvoke(inputs)
        return result.get("generated_answer", "죄송합니다. 오류가 발생했습니다.")
```

### 2.4 Django View 및 URL 설정 (`bot/views.py`)
비동기 뷰(`async def`)를 사용하여 Uvicorn 환경에서 성능을 최적화합니다.

```python
# bot/views.py
from django.http import JsonResponse
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
import json
from .services import ChatbotService

# 메인 채팅 페이지
def index(request):
    return render(request, 'bot/index.html')

# 채팅 API (Async)
async def chat_api(request):
    if request.method == 'POST':
        try:
            data = json.loads(request.body)
            user_message = data.get('message', '')
            
            bot_service = await ChatbotService.get_instance()
            answer = await bot_service.get_response(user_message)
            
            return JsonResponse({'status': 'success', 'answer': answer})
        except Exception as e:
            return JsonResponse({'status': 'error', 'message': str(e)}, status=500)
    return JsonResponse({'status': 'error', 'message': 'Invalid method'}, status=400)
```

### 2.5 프론트엔드 구현 (`templates/bot/index.html`)
간단한 HTML/JS로 채팅 인터페이스를 구성합니다. `fetch` API를 사용하여 비동기로 서버에 메시지를 전송합니다.

```html
<!-- 간단한 채팅 UI 예시 -->
<!DOCTYPE html>
<html>
<head>
    <title>노동법률 AI 챗봇</title>
    <style>
        .chat-container { width: 400px; margin: 0 auto; }
        .chat-box { height: 500px; border: 1px solid #ddd; overflow-y: scroll; padding: 10px; }
        .input-area { margin-top: 10px; display: flex; }
        .input-area input { flex: 1; padding: 10px; }
    </style>
</head>
<body>
    <div class="chat-container">
        <h2>A-TEAM 법률 챗봇</h2>
        <div class="chat-box" id="chatBox"></div>
        <div class="input-area">
            <input type="text" id="userInput" placeholder="질문을 입력하세요..." onkeypress="handleEnter(event)">
            <button onclick="sendMessage()">전송</button>
        </div>
    </div>

    <script>
        async function sendMessage() {
            const input = document.getElementById('userInput');
            const message = input.value;
            if (!message) return;

            // 사용자 메시지 표시
            appendMessage('나', message);
            input.value = '';

            try {
                const response = await fetch('/chat/api/', {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({message: message})
                });
                const data = await response.json();
                if (data.status === 'success') {
                    appendMessage('AI', data.answer);
                } else {
                    appendMessage('System', '오류가 발생했습니다.');
                }
            } catch (e) {
                console.error(e);
            }
        }

        function appendMessage(sender, text) {
            const box = document.getElementById('chatBox');
            const div = document.createElement('div');
            div.innerHTML = `<strong>${sender}:</strong> ${text}`;
            box.appendChild(div);
            box.scrollTop = box.scrollHeight;
        }
        
        function handleEnter(e) {
            if (e.key === 'Enter') sendMessage();
        }
    </script>
</body>
</html>
```

## 3. AWS EC2 배포 계획

### 3.1 인스턴스 생성 및 준비
1. **EC2 인스턴스 시작**:
   - OS: **Ubuntu Server 22.04 LTS** (추천)
   - Instance Type: **t3.medium** 이상 권장 (LLM/임베딩 모델 메모리 고려 필요, 단순 API 호출만 한다면 t3.small도 가능하나 안정성을 위해 medium 추천)
   - Key Pair 생성 및 다운로드 (`.pem`)

2. **보안 그룹(Security Group) 설정**:
   - Inbound Rules:
     - **SSH (22)**: 내 IP에서만 허용
     - **HTTP (80)**: Anywhere (0.0.0.0/0)
     - **HTTPS (443)**: Anywhere (0.0.0.0/0) (향후 SSL 적용 시)
     - **Custom TCP (8000)**: 테스트용 (선택 사항)

### 3.2 서버 환경 설정
SSH로 서버 접속 후 필수 패키지를 설치합니다.

```bash
ssh -i "key.pem" ubuntu@<EC2-Public-IP>

# 시스템 업데이트
sudo apt update && sudo apt upgrade -y

# Python 및 패키지 관리 도구 설치
sudo apt install python3-pip python3-venv nginx git -y
```

### 3.3 프로젝트 배포
1. **코드 업로드**: Git Clone 또는 SCP를 이용해 프로젝트 파일을 서버로 전송합니다.
2. **가상환경 구성**:
```bash
cd ~/project_folder
python3 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```
3. **환경변수 설정**: `.env` 파일을 생성하여 API Key 등을 입력합니다.

### 3.4 Gunicorn + Uvicorn 실행
LangGraph 비동기 처리를 완벽하게 지원하기 위해 Gunicorn의 워커로 Uvicorn을 사용합니다.

```bash
# 테스트 실행
gunicorn legal_chat.asgi:application -k uvicorn.workers.UvicornWorker --bind 0.0.0.0:8000
```

### 3.5 Systemd 서비스 등록 (백그라운드 실행)
리포지토리에 포함된 `deployment/gunicorn_django.service` 파일을 활용합니다.

1. **서비스 파일 수정 및 복사**:
```bash
# 파일 내의 경로(/home/ubuntu/project_folder)를 실제 경로로 수정 후 복사
sudo cp deployment/gunicorn_django.service /etc/systemd/system/
```

2. **실행 및 등록**:
```bash
sudo systemctl start gunicorn_django
sudo systemctl enable gunicorn_django
```

### 3.6 Nginx 설정 (Reverse Proxy)
리포지토리에 포함된 `deployment/nginx_app.conf` 파일을 활용합니다.

1. **설정 파일 수정 및 복사**:
```bash
# 파일 내의 도메인/IP 및 경로 수정 후 복사
sudo cp deployment/nginx_app.conf /etc/nginx/sites-available/legal_chat
```

2. **설정 적용**:
```bash
sudo ln -s /etc/nginx/sites-available/legal_chat /etc/nginx/sites-enabled
sudo nginx -t
sudo systemctl restart nginx
```

## 4. Qdrant 데이터 이전 계획 (DB Migration)

EC2에서 Qdrant를 새로 실행할 경우, 로컬에 저장된 벡터 데이터를 서버로 이전해야 합니다.

### 방법 1: Qdrant Cloud 사용 시 (권장)
별도의 데이터 이전 작업이 필요 없습니다. 로컬과 서버 모두 동일한 클라우드 URL을 바라보게 설정하면 됩니다.
- `.env` 설정:
  ```env
  QDRANT_URL=https://<your-cluster-url>.qdrant.tech
  QDRANT_API_KEY=<your-api-key>
  ```

### 방법 2: 로컬(Docker) -> EC2(Docker) 이전 시
Snapshot 기능을 사용하여 데이터를 통째로 옮깁니다.

#### 1단계: 스냅샷 생성 (로컬)
로컬 터미널에서 실행:
```bash
# 1. 스냅샷 생성 요청
curl -X POST http://localhost:6333/collections/<컬렉션명>/snapshots

# 2. 스냅샷 파일명 확인 후 다운로드
wget http://localhost:6333/collections/<컬렉션명>/snapshots/<스냅샷파일명>.snapshot
```

#### 2단계: 파일 전송 및 복원 (EC2)
1. **EC2로 파일 전송**:
   ```bash
   scp -i key.pem <스냅샷파일명>.snapshot ubuntu@<EC2-IP>:~
   ```
2. **EC2에서 복원**:
   ```bash
   # EC2 접속 후
   curl -X POST -F "snapshot=@<스냅샷파일명>.snapshot" \
        http://localhost:6333/collections/<컬렉션명>/snapshots/upload
   ```

## 5. 추가 고려사항
1. **HTTPS 적용**: Certbot(Let's Encrypt)을 사용하여 무료 SSL 인증서를 발급받아 보안을 강화합니다.
2. **비동기 타임아웃**: LLM 응답이 늦어질 수 있으므로 Nginx의 `proxy_read_timeout` 설정을 넉넉하게 잡습니다 (예: 300초).
3. **비용 관리**: EC2 미사용 시 중지하거나, Elastic IP를 사용하여 IP 변경을 방지합니다.

이 계획을 바탕으로 단계별 구현을 시작하시면 됩니다. 궁금한 점이 있다면 언제든 물어봐주세요!